# Summary of notebook
Author: Peter Ferguson (peter.ferguson@wisc.edu)
Last run: 2023/04/11

This notebook along with the file `create_slew_profiles.py` are used in this analysis. 


1. The file `create_slew_profiles.py` is called to query the EFD mount encoder data over the specified time window, identify slews in this time window and model the slew profiles with splines. Then the results are saved to parquet tables (output directory specified on line 572). 
    - example call `python create_slew_profiles.py -d 2023-03-23 -w 10 -m 1`
    - d is the date of the start of the window
    - w is the lengh of the window in hours
    - t gives the exact time of the window start default is "14:01:00"
    - sm specifies the method used to identify slews. 
        - Method 1 is more reliable and uses the command track and inPosition logevents
        - Method 2 only uses mount position encoder data to identify slews
    - fm specifies the method used to fit the slews and can be `splines` or `savgol`
    - f force boolean specifies weather script should be run if output files already exist
2. This notebook runs `create_slew_profiles.py`, then takes the results and identifies the maximum velocity/acceleration/jerk for each slew.
    - Plots the date distribution of identified slews 
    - Then plots a histograms showing all of these max values
    - identifies any slews that exceed design/max limits. 
    - provides some functionality to create slew profile plots for any slews that are found to exceed limits (so far all of these have been due to problems in the spline fit not slews exceeding specs)


# Imports

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
import pandas as pd
import glob
import datetime
from scipy.interpolate import interp1d
import subprocess
import os
import numpy as np
from lsst.sitcom import vandv


In [ ]:
#setup plotting
params = {'xtick.direction': 'in',
          'ytick.direction': 'in',
          }

plt.rcParams.update(params)
# limits from srd all units are deg/s - deg/s^2 - deg/s^3
el_limit_dict={
    "max_velocity": 5.25,
    "max_acceleration": 5.25,
    "max_jerk": 21,
    "design_velocity": 3.5,
    "design_acceleration": 3.5,
    "design_jerk": 14,
}
az_limit_dict={
    "max_velocity": 10.5,
    "max_acceleration": 10.5,
    "max_jerk": 42,
    "design_velocity": 7,
    "design_acceleration": 7,
    "design_jerk": 28,
}

In [ ]:
# setup query 
query_duration = 24 # hours
slew_identification = 1 # reccomended 1 use log events to identify slews
fit_method = "splines"
force = False



date_min=59876 # this takes you to begining of november 2022
date_max=60033.0
mjds=np.arange(date_max,date_min, -1)
days=[Time(i, format='mjd', scale="utc").iso[:10] for i in mjds]

# to only run quieries on the days with known data uncomment this portion
days=[
   # '2022-11-03',
   #  '2022-11-09',
   #  '2022-11-10',
   #  '2022-11-21',
   #  '2022-11-22',
   #  '2022-11-24',
    '2023-01-30',
    '2023-01-31',
    '2023-03-15',
    '2023-03-16',
    '2023-03-17',
    '2023-03-20',
    '2023-03-21',
    '2023-03-22',
    '2023-03-23',
    '2023-03-24',
    '2023-03-27',
    '2023-03-28',]

In [ ]:
# query EFD and create slew profiles this can either be done over a range, or we have a list of known dates

for day in days:
    cmd= f"python create_slew_profiles.py -d {day} -w {query_duration} " 
    cmd+= f"-sm {slew_identification} -fm {fit_method}"
    if force:
        cmd+=" -f"
    print(cmd)
    cmd=cmd.split()

    subprocess.run(cmd)

In [ ]:
# the max_slew_rates_python/create_slew_profiles.py code must be run first to create the spline_frames that are read in here.
# choose method 1 or 2. recommended to use method 1
method=slew_identification
fileList=glob.glob(f"./data/method_{slew_identification}/{fit_method}*.parquet")
spline_frame=[]
for file in fileList:
    if "" in file:
        spline_frame.append(pd.read_parquet(file))
spline_frame=pd.concat(spline_frame)

In [ ]:
# identify maximums for each slew
az_vel_max = []
el_vel_max = []

az_acc_max = []
el_acc_max = []

az_jerk_max = []
el_jerk_max = []
slew_num = []
day_num = []
slew_time= []
for i in np.unique(spline_frame["day"]):
    seld=(spline_frame["day"]==i)
    for j in np.unique(spline_frame["slew_index"][seld]):
        sel=seld & (spline_frame["slew_index"]==j)
        az_vel_max.append(abs(spline_frame.loc[sel,"azVelocity"]).max())
        el_vel_max.append(abs(spline_frame.loc[sel,"elVelocity"]).max())

        az_acc_max.append(abs(spline_frame.loc[sel,"azAcceleration"]).max())
        el_acc_max.append(abs(spline_frame.loc[sel,"elAcceleration"]).max())

        az_jerk_max.append(abs(spline_frame.loc[sel,"azJerk"]).max())
        el_jerk_max.append(abs(spline_frame.loc[sel,"elJerk"]).max())
        slew_num.append(j)
        day_num.append(i)
        slew_time.append(np.max([spline_frame.loc[sel,"azTime"].max(),spline_frame.loc[sel,"elTime"].max()]))
        #if spline_frame.loc[sel,"elAcceleration"].max() >
max_frame=pd.DataFrame({
    "day":day_num,
    "slew":slew_num,
    "az_vel":az_vel_max,
    "az_acc":az_acc_max,
    "az_jerk":az_jerk_max,
    "el_vel":el_vel_max,
    "el_acc":el_acc_max,
    "el_jerk":el_jerk_max})

In [ ]:
start=datetime.datetime(2022, 11, 1, 0, 0)
stop=Time(day_num, format='isot').datetime.max() + datetime.timedelta(days=2)
datebins=np.arange(start,stop,datetime.timedelta(days=1))
fig, axs = plt.subplots(1, dpi=175, figsize=(10,5))
ax=axs
#ax.plot(Time(day, format='isot').datetime,nslews)
ax.hist(Time(day_num, format='isot').datetime, bins=datebins )
plt.setp( ax.xaxis.get_majorticklabels(), rotation=15 )
ax.set_xlabel("Date")
ax.set_ylabel("N-slews")
ax.set_title(f"Date Distribution of Slews\nTotal Slews: {len(day_num)}")
# ax=axs[1]
# ax.hist(np.array(slew_time)-8, bins=100, range=[0,100])
# ax.set_yscale('log')
# ax.set_xlabel('Slew time [s]')
plt.savefig(f"./date_hist_method_{method}_{fit_method}.png")

In [ ]:

# az_sel=(max_frame["az_vel"] > 0)
# el_sel=(max_frame["el_vel"] > 0.05)
velmax=12
accmax=12
jerkmax=45
velbins=np.linspace(0,velmax, 100)
accbins=np.linspace(0,accmax, 100)
jerkbins=np.linspace(0,jerkmax, 100)
nslew=len(np.array(max_frame))

fig, axs = plt.subplots(2,3,figsize=(10,7), dpi=175)

ax=axs[0,0]
ax.set_title("Max Velocity")
azVelHist=ax.hist(max_frame["az_vel"], color="tab:blue", bins=velbins)
ax.axvline(az_limit_dict["design_velocity"], color="tab:orange",ls="--", label="Design")
ax.axvline(az_limit_dict["max_velocity"], color="tab:red", ls="--", label="Max")
ax.set_ylabel("Azmiuth [Count]")
ax.set_xlim(0, velmax)
ax.set_xticklabels([])
#axs[0,0].set_yscale('log')

ax=axs[0,1]
ax.set_title("Max Acceleration")
azAccHist=ax.hist(max_frame["az_acc"], color="tab:blue", bins=accbins)
ax.axvline(az_limit_dict["design_acceleration"], color="tab:orange",ls="--", label="Design")
ax.axvline(az_limit_dict["max_acceleration"], color="tab:red", ls="--", label="Max")
ax.set_xlim(0, accmax)
ax.set_xticklabels([])
ax.set_yticklabels([])


ax=axs[0,2]
ax.set_title("Max Jerk")
azJerkHist=ax.hist(max_frame["az_jerk"], color="tab:blue", bins=jerkbins)
ax.axvline(az_limit_dict["design_jerk"], color="tab:orange",ls="--", label="Design")
ax.axvline(az_limit_dict["max_jerk"], color="tab:red", ls="--", label="Max")
ax.set_xlim(0, jerkmax)
ax.legend(loc=1, framealpha=1, edgecolor="white")
ax.set_xticklabels([])
ax.set_yticklabels([])



ax=axs[1,0]
elVelHist=ax.hist(max_frame["el_vel"], color="tab:green", bins=velbins)
ax.axvline(el_limit_dict["design_velocity"], color="tab:orange",ls="--", label="Design", alpha=0.8)
ax.axvline(el_limit_dict["max_velocity"], color="tab:red", ls="--", label="Max")

ax.set_xlabel("Velocity [deg/s]")
ax.set_ylabel("Elevation [Count]")
ax.set_xlim(0, velmax)
ax.set_xticks(np.arange(0,12,2))


ax=axs[1,1]
elAccHist=ax.hist(max_frame["el_acc"], color="tab:green", bins=accbins)
ax.axvline(el_limit_dict["design_acceleration"], color="tab:orange",ls="--", label="Design")
ax.axvline(el_limit_dict["max_acceleration"], color="tab:red", ls="--", label="Max")
ax.set_xlim(0, accmax)
ax.set_xlabel("Acceleration [deg/s$^2$]")
ax.set_yticklabels([])
ax.set_xticks(np.arange(0,12,2))

ax=axs[1,2]
elJerkHist=ax.hist(max_frame["el_jerk"], color="tab:green", bins=jerkbins)
ax.axvline(el_limit_dict["design_jerk"], color="tab:orange",ls="--", label="Design")
ax.axvline(el_limit_dict["max_jerk"], color="tab:red", ls="--", label="Max")

ax.set_xlim(0, jerkmax)
ax.set_xlabel("Jerk [deg/s$^3$]")
ax.set_yticklabels([])


plt.subplots_adjust(wspace=0, hspace=0)

az_yax_max=np.max([azVelHist[0].max(),azAccHist[0].max(), azJerkHist[0].max()]) *1.1
el_yax_max=np.max([elVelHist[0].max(),elAccHist[0].max(), elJerkHist[0].max()]) *1.1
for i in range(3):
    axs[0,i].set_ylim(0,az_yax_max)
    axs[1,i].set_ylim(0,el_yax_max)

#axs[0,1].set_yscale('log')

#az_yax_max=np.max([azVelHist[0],azAccHist[0], axJerkHist[0]]) *1.2
#print(az_yax_max)
#axs[0,0].set_ylim(1,az_yax_max)
# 
title=f"MT Mount Vels Accels and Jerks, {nslew} slews\n2022/11/03 -- 2023/03/25"
plt.suptitle(title, fontsize = 13, y=0.97)
plt.savefig(f"./all_slews_max_method_{method}_{fit_method}.png", facecolor="white")

In [ ]:
print_dict={'az_vel': 'Az Velocity    ', 
            'az_acc': 'Az Acceleration', 
            'az_jerk':'Az Jerk        ', 
            'el_vel': 'El Velocity    ', 
            'el_acc': 'El Acceleration', 
            'el_jerk':'El Jerk        ',
           }
limit_trans={'vel': 'velocity',
             'acc': 'acceleration',
             'jerk': 'jerk'}
print(f"{len(max_frame)} slews     "+ "| > design |" + " > max    |")    
for k in print_dict.keys():
    # k_checked_dict=az_checked_dict
    sel=(max_frame["day"])
    if k[:2] == 'el':
        pstring = print_dict[k] 
        pstring+= f"| {(max_frame[k] > el_limit_dict['design_'+limit_trans[k[3:]]]).sum():<5}    |"
        pstring+= f" {(max_frame[k] > el_limit_dict['max_'+limit_trans[k[3:]]]).sum():<5}    |"
        print(pstring )
    if k[:2] == 'az':
        pstring = print_dict[k] 
        pstring+= f"| {(max_frame[k] > az_limit_dict['design_'+limit_trans[k[3:]]]).sum():<5}    |"
        pstring+= f" {(max_frame[k] > az_limit_dict['max_'+limit_trans[k[3:]]]).sum():<5}    |"
        print(pstring )

In [ ]:
k="az_jerk"
max_frame[max_frame[k] > az_limit_dict['max_'+limit_trans[k[3:]]]]

In [ ]:
# section is to read in the raw encoder data
azFileList=glob.glob(f"./data/method_{method}/az_measurement_frame*.parquet")
az_measurement_frame=[]
for file in azFileList:
    az_measurement_frame.append(pd.read_parquet(file))
az_measurement_frame=pd.concat(az_measurement_frame)

elfileList=glob.glob(f"./data/method_{method}/el_measurement_frame*.parquet")
el_measurement_frame=[]
for file in elfileList:
    el_measurement_frame.append(pd.read_parquet(file))
el_measurement_frame=pd.concat(el_measurement_frame)

In [ ]:
# create slew profiles for every slew that exceeds the k limit
k="az_"+ "jerk" # "el_", "acc","vel" # k gives the parameter to check against limits
lt="design" # limit type can be design or max
outdir=f"./plots/az_checks/{k}_{lt}" # plot output directory 
plotnum=0
os.makedirs(outdir, exist_ok=True)
submax=max_frame[max_frame[k] > az_limit_dict[f'{lt}_'+limit_trans[k[3:]]]]
for day in np.unique(submax["day"]):
    seld=(submax["day"]==day)
    for slew in np.unique(submax["slew"][seld]):
        if plotnum < 20:
            plotnum +=1
            print(day, slew)
            sel=(spline_frame["day"]==day)& (spline_frame["slew_index"]==slew)

            sel_az=(az_measurement_frame["day"]==day) & (az_measurement_frame["slew_index"]==slew)
            fig,ax=plt.subplots(4, figsize=(10,6))
            title="Azimuth\n"+Time(np.unique(spline_frame["azZeroTime"][sel])[0], format='unix').iso + f"\n day: {day} slew number {slew}"
            plt.subplot(411)
            plt.title(title)
            plt.plot(spline_frame["azTime"][sel], spline_frame["azVelocity"][sel], color="k")
            plt.scatter(az_measurement_frame["az_times"][sel_az], az_measurement_frame["azVelocityMeas"][sel_az], s=5, c="tab:green", label="data")

            limtype="velocity"
            plt.axhline(az_limit_dict[f"max_{limtype}"], c="red", label="max")
            plt.axhline(-1 * az_limit_dict[f"max_{limtype}"], c="red")
            plt.axhline(az_limit_dict[f"design_{limtype}"], c="tab:orange", label="max")
            plt.axhline(-1 * az_limit_dict[f"design_{limtype}"], c="tab:orange")

            plt.ylabel("velocity")
            plt.subplot(412)

            vel_interp=interp1d(spline_frame["azTime"][sel], spline_frame["azVelocity"][sel], bounds_error=False)
            meas_resid=az_measurement_frame["azVelocityMeas"][sel_az] - vel_interp(az_measurement_frame["az_times"][sel_az])
            plt.scatter(az_measurement_frame["az_times"][sel_az], meas_resid, s=5, c="tab:green", label="data")
            plt.ylabel("residual")
            resid_lim=np.max([np.max(abs(meas_resid)), 1])
            plt.ylim(-resid_lim, resid_lim)

            plt.subplot(413)
            plt.plot(spline_frame["azTime"][sel], spline_frame["azAcceleration"][sel], color="k")
            limtype="acceleration"
            plt.axhline(az_limit_dict[f"max_{limtype}"], c="red", label="max")
            plt.axhline(-1 * az_limit_dict[f"max_{limtype}"], c="red")
            plt.axhline(az_limit_dict[f"design_{limtype}"], c="tab:orange", label="max")
            plt.axhline(-1 * az_limit_dict[f"design_{limtype}"], c="tab:orange")

            plt.ylabel("Acceleration")
            plt.subplot(414)
            plt.plot(spline_frame["azTime"][sel], spline_frame["azJerk"][sel], color="k")

            limtype="jerk"
            plt.axhline(az_limit_dict[f"max_{limtype}"], c="red", label="max")
            plt.axhline(-1 * az_limit_dict[f"max_{limtype}"], c="red")
            plt.axhline(az_limit_dict[f"design_{limtype}"], c="tab:orange", label="max")
            plt.axhline(-1 * az_limit_dict[f"design_{limtype}"], c="tab:orange")
            plt.ylabel("Jerk")
            plt.xlabel("time [s]")
            plt.savefig(outdir+f"/{day}_{slew}_{k}_check.png", facecolor="white")
            plt.close()

In [ ]:
k="el_jerk"
lt="max"
outdir=f"./plots/el_checks/{k}_{lt}"
plotnum=0
os.makedirs(outdir, exist_ok=True)
submax=max_frame[max_frame[k] > el_limit_dict[f'{lt}_'+limit_trans[k[3:]]]]
for day in np.unique(submax["day"]):
    seld=(submax["day"]==day)
    for slew in np.unique(submax["slew"][seld]):
        if plotnum < 20:
            plotnum +=1
            print(day, slew)
            sel=(spline_frame["day"]==day)& (spline_frame["slew_index"]==slew)

            sel_el=(el_measurement_frame["day"]==day) & (el_measurement_frame["slew_index"]==slew)
            fig,ax=plt.subplots(4, figsize=(10,6))
            title="Elevation\n"+Time(np.unique(spline_frame["elZeroTime"][sel])[0], format='unix').iso + f"\n day: {day} slew number {slew}"
            plt.subplot(411)
            plt.title(title)
            plt.plot(spline_frame["elTime"][sel], spline_frame["elVelocity"][sel], color="k")
            plt.scatter(el_measurement_frame["el_times"][sel_el], el_measurement_frame["elVelocityMeas"][sel_el], s=5, c="tab:green", label="data")

            limtype="velocity"
            plt.axhline(el_limit_dict[f"max_{limtype}"], c="red", label="max")
            plt.axhline(-1 * el_limit_dict[f"max_{limtype}"], c="red")
            plt.axhline(el_limit_dict[f"design_{limtype}"], c="tab:orange", label="max")
            plt.axhline(-1 * el_limit_dict[f"design_{limtype}"], c="tab:orange")

            plt.ylabel("velocity")
            plt.subplot(412)

            vel_interp=interp1d(spline_frame["elTime"][sel], spline_frame["elVelocity"][sel], bounds_error=False)
            meas_resid=el_measurement_frame["elVelocityMeas"][sel_el] - vel_interp(el_measurement_frame["el_times"][sel_el])
            plt.scatter(el_measurement_frame["el_times"][sel_el], meas_resid, s=5, c="tab:green", label="data")
            plt.ylabel("residual")
            resid_lim=np.max([np.max(abs(meas_resid)), 1])
            plt.ylim(-resid_lim, resid_lim)

            plt.subplot(413)
            plt.plot(spline_frame["elTime"][sel], spline_frame["elAcceleration"][sel], color="k")
            limtype="acceleration"
            plt.axhline(el_limit_dict[f"max_{limtype}"], c="red", label="max")
            plt.axhline(-1 * el_limit_dict[f"max_{limtype}"], c="red")
            plt.axhline(el_limit_dict[f"design_{limtype}"], c="tab:orange", label="max")
            plt.axhline(-1 * el_limit_dict[f"design_{limtype}"], c="tab:orange")


            plt.ylabel("Acceleration")
            plt.subplot(414)
            plt.plot(spline_frame["elTime"][sel], spline_frame["elJerk"][sel], color="k")

            limtype="jerk"
            plt.axhline(el_limit_dict[f"max_{limtype}"], c="red", label="max")
            plt.axhline(-1 * el_limit_dict[f"max_{limtype}"], c="red")
            plt.axhline(el_limit_dict[f"design_{limtype}"], c="tab:orange", label="max")
            plt.axhline(-1 * el_limit_dict[f"design_{limtype}"], c="tab:orange")

            plt.ylabel("Jerk")
            plt.xlabel("time [s]")
            plt.savefig(outdir+f"/{day}_{slew}_{k}_check.png", facecolor="white")
            plt.close()